# Metadaten laden

Die Metadaten enthalten informationen zur Position und Namen der Station. Diese haben sich über die Zeit leicht verändert. Daher muss dies später beim Verwenden der Daten berücksichtigt werden. Zudem helfen die Metadaten, nach Bundesland zu filtern.

In [1]:
import requests
from bs4 import BeautifulSoup
import os

META_DATA_URL = "https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/10_minutes/wind/meta_data/"

TARGET_FOLDER = os.path.join("..", "data", "raw", "station_meta")

TARGET_UNPACKED_FOLDER = os.path.join("..", "data", "interim", "station_meta")

In [2]:

def get_meta_data_url_list():
    '''
    Die Metadaten liegen in verschiedenen Dateien, die über verschiedene URLs zur Verfügung gestellt werden. Die Liste der URLs wird hier erstellt.
    :return: 
    '''
    r = requests.get(META_DATA_URL)
    r.raise_for_status()
    # get all urls from the html with the ending .zip
    soup = BeautifulSoup(r.text, "html.parser")
    urls = [META_DATA_URL+ a["href"] for a in soup.find_all("a") if a["href"].endswith(".zip")]
    return urls

urls = get_meta_data_url_list()

for url in urls:
    filename = url.split("/")[-1]
    target_path = os.path.join(TARGET_FOLDER, filename)
    if not os.path.exists(target_path):
        print(f"Downloading {url} to {target_path}")
        response = requests.get(META_DATA_URL + url)
        with open(target_path, "wb") as f:
            f.write(response.content)
    else:
        print(f"Skipping {url} as {target_path} already exists")